Suppose we live at a place where the weather is either sunny, rainy and cloudy. Suppose that we cannot observe the weather directly, but instead rely on a sensor. The problem is that our sensor is noisy. Its measurements are governed by the following measurement model:

If actual weather is sunny, the sensor tell us
Sunny: 0.6
Cloudy: 0.4
Rainy: 0.0

If actual weather is cloudy, the sensor tell us
Sunny: 0.3
Cloudy: 0.7
Rainy: 0.0

If actual weather is rainy, the sensor tell us
Sunny: 0.0
Cloudy: 0.0
Rainy: 1.0

Suppose Day 1 is sunny (this is known for a fact), and in the subsequent four days our 
sensor observes cloudy, cloudy, rainy, sunny. What is the probability that Day 5 is 
indeed sunny as predicted by our sensor ?

In [1]:
import numpy as np

# Sensor measurement model
measurement_model = np.array([[0.6, 0.4, 0.0],
                              [0.3, 0.7, 0.0],
                              [0.0, 0.0, 1.0]])

# Initial probabilities
initial_probabilities = np.array([1.0, 0.0, 0.0])

# Transition matrix
transition_matrix = np.array([[0.8, 0.2, 0.0],
                              [0.4, 0.4, 0.2],
                              [0.2, 0.6, 0.2]])

# Observation sequence
observation_sequence = ['Cloudy', 'Cloudy', 'Rainy', 'Sunny']

# Convert observation sequence to corresponding indices
observations = [['Sunny', 'Cloudy', 'Rainy'].index(obs) for obs in observation_sequence]

# Perform the forward algorithm
forward_probabilities = np.zeros((len(observation_sequence) + 1, 3))
forward_probabilities[0] = initial_probabilities

for t in range(1, len(observation_sequence) + 1):
    for j in range(3):
        for i in range(3):
            forward_probabilities[t, j] += (forward_probabilities[t - 1, i] *
                                            transition_matrix[i, j] *
                                            measurement_model[j, observations[t - 1]])

# Calculate the probability of Day 5 being sunny
probability_sunny = forward_probabilities[-1, 0] / np.sum(forward_probabilities[-1])

# Print the probability
print("Probability that Day 5 is sunny: {:.3f}".format(probability_sunny))


Probability that Day 5 is sunny: 0.400


Once again, suppose Day 1 is known to be sunny. At Days 2 through 4, the sensor measures sunny, sunny, rainy. For each of the Days 2 through 4, what is the most likely weather on that day ? Answer the question in two ways: one in which only the data available to the day in question is used, and one in hindsight, where data from future days is also available.

Let's calculate the most likely weather for each of the Days 2 through 4 using:

Using only the available data:

In [2]:
# Observations
observations = ['Sunny', 'Sunny', 'Rainy']

# Convert observations to corresponding indices
obs_indices = [['Sunny', 'Cloudy', 'Rainy'].index(obs) for obs in observations]

# Perform the Viterbi algorithm
num_states = transition_matrix.shape[0]
num_obs = len(obs_indices)

# Initialize matrices
viterbi = np.zeros((num_obs, num_states))
backpointers = np.zeros((num_obs, num_states), dtype=int)

# Base case: Day 1
viterbi[0] = initial_probabilities * measurement_model[:, obs_indices[0]]

# Recursive case: Days 2 through 4
for t in range(1, num_obs):
    for j in range(num_states):
        prev_viterbi = viterbi[t - 1] * transition_matrix[:, j]
        viterbi[t, j] = np.max(prev_viterbi) * measurement_model[j, obs_indices[t]]
        backpointers[t, j] = np.argmax(prev_viterbi)

# Find the most likely weather for each day
most_likely_weather = [np.argmax(viterbi[t]) for t in range(num_obs)]

# Convert indices back to weather conditions
weather_conditions = ['Sunny', 'Cloudy', 'Rainy']

# Print the most likely weather for each day
for t in range(num_obs):
    day = t + 2
    print("Day {}: {}".format(day, weather_conditions[most_likely_weather[t]]))

Day 2: Sunny
Day 3: Sunny
Day 4: Rainy


Using data in hindsight:

In [3]:
# Observations
observations = ['Sunny', 'Sunny', 'Rainy']

# Convert observations to corresponding indices
obs_indices = [['Sunny', 'Cloudy', 'Rainy'].index(obs) for obs in observations]

# Perform the forward algorithm
forward_probabilities = np.zeros((len(obs_indices), 3))
forward_probabilities[0] = initial_probabilities * measurement_model[:, obs_indices[0]]

for t in range(1, len(obs_indices)):
    for j in range(3):
        forward_probabilities[t, j] = np.sum(forward_probabilities[t - 1] * transition_matrix[:, j]) * measurement_model[j, obs_indices[t]]

# Perform the backward algorithm
backward_probabilities = np.ones((len(obs_indices), 3))

for t in range(len(obs_indices) - 2, -1, -1):
    for i in range(3):
        backward_probabilities[t, i] = np.sum(backward_probabilities[t + 1] * transition_matrix[i, :] * measurement_model[:, obs_indices[t + 1]])

# Calculate the smoothed probabilities
smoothed_probabilities = forward_probabilities * backward_probabilities
smoothed_probabilities /= np.sum(smoothed_probabilities, axis=1, keepdims=True)

# Find the most likely weather for each day
most_likely_weather = np.argmax(smoothed_probabilities, axis=1)

# Convert indices back to weather conditions
weather_conditions = ['Sunny', 'Cloudy', 'Rainy']

# Print the most likely weather for each day
for t in range(len(obs_indices)):
    day = t + 2
    print("Day {}: {}".format(day, weather_conditions[most_likely_weather[t]]))


Day 2: Sunny
Day 3: Cloudy
Day 4: Rainy


Consider the same situation (Day 1 is sunny, the measurements for Days 2, 3, and 4 are  sunny, sunny, rainy). What is the most likely sequence of weather for Days 2 through  4? What is the probability of this most likely sequence ?

In [4]:
# Observations
observations = ['Sunny', 'Sunny', 'Rainy']

# Convert observations to corresponding indices
obs_indices = [['Sunny', 'Cloudy', 'Rainy'].index(obs) for obs in observations]

# Perform the Viterbi algorithm
num_states = transition_matrix.shape[0]
num_obs = len(obs_indices)

# Initialize matrices
viterbi = np.zeros((num_obs, num_states))
backpointers = np.zeros((num_obs, num_states), dtype=int)

# Base case: Day 1
viterbi[0] = initial_probabilities * measurement_model[:, obs_indices[0]]

# Recursive case: Days 2 through 4
for t in range(1, num_obs):
    for j in range(num_states):
        prev_viterbi = viterbi[t - 1] * transition_matrix[:, j]
        viterbi[t, j] = np.max(prev_viterbi) * measurement_model[j, obs_indices[t]]
        backpointers[t, j] = np.argmax(prev_viterbi)

# Find the most likely sequence
most_likely_sequence = [np.argmax(viterbi[t]) for t in range(num_obs)]

# Convert indices back to weather conditions
weather_conditions = ['Sunny', 'Cloudy', 'Rainy']
most_likely_weather = [weather_conditions[state] for state in most_likely_sequence]

# Calculate the probability of the most likely sequence
probability = np.max(viterbi[-1])

# Print the most likely sequence and its probability
print("Most likely sequence of weather: {}".format(most_likely_weather))
print("Probability of the most likely sequence: {:.3f}".format(probability))

Most likely sequence of weather: ['Sunny', 'Sunny', 'Rainy']
Probability of the most likely sequence: 0.007
